In [1]:
%run util.py

/home/jzh/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
data_folder = '/home/jzh/Waze/'

In [3]:
import csv
import json

In [4]:
# Loading JSON data
with open(data_folder + 'points_dict_Mar16.json', 'r') as json_file:
    points_dict = json.load(json_file)

### Extract jam data for a typical workday, say, March 9, 2016 (Wednesday)
The features we are going to use are: 
(longitude, latitude), numPts, speed, length, startTime.

Note that "(longitude, latitude), numPts" are from the points data.

Convert "startTime" to seconds, discarding the date stamp.

In [5]:
from geopy.distance import vincenty

##### cf. http://www.latlong.net/
BU = (42.351848, -71.110730)

with open(data_folder + 'raw_data/waze_jams.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    i = 0
    jam_ref_data = {}
    
    for row in reader:
#         print(row.keys())
        # print(row['street'])
        if ('2016-03-09' in row['startTime']):
            pt = (points_dict[row['uuid']][0][0][1], points_dict[row['uuid']][0][0][0])
            # If the number of points in a jam is less than 20, we roughly take this jam as "normal"
            if (vincenty(BU, pt).meters < 3e3 and points_dict[row['uuid']][1] < 20):
                key = str(i)
                data = {'uuid': row['uuid'],
                        'longitude_latitude': points_dict[row['uuid']][0],
                        'numPts': points_dict[row['uuid']][1], 
                        'speed': float(row['speed']), 
                        'length': float(row['Length']), 
                        'startTime': convStartTimeToSec(row['startTime'])}
                jam_ref_data[key] = data
                i += 1
print(i)

2829


In [6]:
# Writing JSON data
with open(data_folder + 'jam_ref_data.json', 'w') as json_file:
    json.dump(jam_ref_data, json_file)